# Part 3: Data Analytics

## Step 0

> 0. Load both the csv file from **Part 1** `pr.data.0.current` and the json file from **Part 2** as dataframes ([Spark](https://spark.apache.org/docs/1.6.1/api/java/org/apache/spark/sql/DataFrame.html), [Pyspark](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.html), [Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html), [Koalas](https://koalas.readthedocs.io/en/latest/), etc).

In [1]:
# Imports
import pandas as pd
import requests

# Data sources
SERIES_DATA = "https://data-quest-bucket.s3.amazonaws.com/pr.data.0.Current"
POPULATION_DATA = "https://data-quest-bucket.s3.amazonaws.com/population-data.json"

# Load data
series = pd.read_csv(SERIES_DATA, delimiter="\t")
r = requests.get(POPULATION_DATA).json()
population = pd.json_normalize(r, record_path="data")

## Step 1

> 1. Using the dataframe from the population data API (Part 2), generate the mean and the standard deviation of the US population across the years [2013, 2018] inclusive.

In [2]:
# Filter data between [2013, 2018] inclusive
population_stats = population[(population["Year"].astype(int) >= 2013) &
                              (population["Year"].astype(int) <= 2018)]
# Display mean and standard deviation of population
population_stats["Population"].mean(), population_stats["Population"].std()

(322069808.0, 4158441.040908095)

## Step 2

> Using the dataframe from the time-series (Part 1), For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year.

In [3]:
# Remove columns names whitespace
series.rename(columns={"series_id        ": "series_id"}, inplace=True)
series.rename(columns={"       value": "value"}, inplace=True)
# Generate report
max_value_series = series.groupby(["series_id", "year"], as_index=False)["value"].agg("sum")
max_value_series = max_value_series.sort_values("value", ascending=False).drop_duplicates("series_id", keep="first").sort_index().reset_index(drop=True)
max_value_series

,series_id,year,value
0,PRS30006011,2012,9.500
1,PRS30006012,2021,13.100
2,PRS30006013,1998,733.290
3,PRS30006021,2010,14.200
4,PRS30006022,2010,11.100
...,...,...,...
277,PRS88003192,2002,285.200
278,PRS88003193,2021,620.416
279,PRS88003201,2007,21.700
280,PRS88003202,2021,26.500


## Step 3

> Using both dataframes from Part 1 and Part 2, generate a report that will provide the `value` for `series_id = PRS30006032` and `period = Q01` and the `population` for that given year (if available in the population dataset)

In [4]:
# Filter series data
specific_series = series.loc[series["series_id"].str.contains("PRS30006032", case=False)]
specific_series = specific_series[specific_series["period"].str.contains("Q01", case=False)]
# Filter population data
population_extract = population["Population"].astype(int)
population["Year"] = population["Year"].astype(int)
# Merge and filter results
result = pd.merge(specific_series, population, left_on="year", right_on="Year", how="left")
result[["series_id", "year", "period", "value", "Population"]]

,series_id,year,period,value,Population
0,PRS30006032,1995,Q01,0.0,NaN
1,PRS30006032,1996,Q01,-4.4,NaN
2,PRS30006032,1997,Q01,2.7,NaN
3,PRS30006032,1998,Q01,1.0,NaN
4,PRS30006032,1999,Q01,-4.1,NaN
5,PRS30006032,2000,Q01,0.1,NaN
6,PRS30006032,2001,Q01,-6.0,NaN
7,PRS30006032,2002,Q01,-7.0,NaN
8,PRS30006032,2003,Q01,-5.7,NaN
9,PRS30006032,2004,Q01,2.4,NaN


## Step 4

> Submit your analysis, your queries, and the outcome of the reports as a [.ipynb](https://github.com/rearc-data/quest#:~:text=Submit%20your%20analysis%2C%20your%20queries%2C%20and%20the%20outcome%20of%20the%20reports%20as%20a%20.ipynb%20file.) file.

`data_analytics.ipynb` is located in the `part3` directory.